## Library importing

In [8]:

from operator import add
from typing import List, TypedDict, Optional, Annotated, Dict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END

In [22]:
def call_agent(state):
    print(state)
    return {"result":"model calling"}

def call_tool(state):
    print(state)
    return {"calling":"model calling"}

def should_continue(state):
    print(state)
    
    return "final"
    # return {"calling":"model calling"}


class GraphStat(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    calling: str
    generation: str
    result: str

In [120]:
from typing import TypedDict

def call_agent(state):
    print("this is action plan",state)
    number_iter = state.get("number_iter",1)
    if number_iter==None:
        number_iter = 1
    else:
        number_iter+=1
    
    final_result = 0
    if state["result"]:
        final_result = state["result"]*2
    else:
        final_result = state["question"]*2
    return {
        "result": final_result,
        "number_iter":number_iter
    }

def call_action(state):
    # state["number_iter"] = state.get("number_iter",1)
    
    return {
        "calling": f"tool calling {state["number_iter"]}",
    }

def call_final(state):
    # state["number_iter"] = 0
    return {
        "calling": f"call_tool calling {state["number_iter"]}"}

def should_continue(state):
    
    if state["number_iter"]>10:
        return 'e'
    
    elif state["result"]<=10:
        return "c"
    
    elif state["result"]<100:
        return "f"

    return "e"  

def final_call_continue_need(state):
    
    if state["number_iter"]>5:
        return "e"
    else:
        return "a"
    
    
class GraphStat(TypedDict):
    calling: str = ""
    question: int=0
    result: int 
    number_iter:int = 0

# Initialize StateGraph with initial state or appropriate parameters
workflow = StateGraph(GraphStat)

workflow.add_node("agent", call_agent)
workflow.add_node("action", call_action)
workflow.add_node("final", call_final)
# workflow.add_node("end", lambda state: {"calling":"not found"})

workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "c": "action", 
        "f": "final", 
        "e": END
        },
)

workflow.add_edge("action", "final")  
workflow.add_conditional_edges("final",final_call_continue_need,
                               {
        "a": "agent", 
        "e": END
        },)

initial_state = {"question":5}
app = workflow.compile()

for data in app.stream(initial_state):
    print("==>",data)




this is action plan {'calling': None, 'question': 5, 'result': None, 'number_iter': None}
==> {'agent': {'result': 10, 'number_iter': 1}}
==> {'action': {'calling': 'tool calling 1'}}
==> {'final': {'calling': 'call_tool calling 1'}}
this is action plan {'calling': 'call_tool calling 1', 'question': 5, 'result': 10, 'number_iter': 1}
==> {'agent': {'result': 20, 'number_iter': 2}}
==> {'final': {'calling': 'call_tool calling 2'}}
this is action plan {'calling': 'call_tool calling 2', 'question': 5, 'result': 20, 'number_iter': 2}
==> {'agent': {'result': 40, 'number_iter': 3}}
==> {'final': {'calling': 'call_tool calling 3'}}
this is action plan {'calling': 'call_tool calling 3', 'question': 5, 'result': 40, 'number_iter': 3}
==> {'agent': {'result': 80, 'number_iter': 4}}
==> {'final': {'calling': 'call_tool calling 4'}}
this is action plan {'calling': 'call_tool calling 4', 'question': 5, 'result': 80, 'number_iter': 4}
==> {'agent': {'result': 160, 'number_iter': 5}}
